In [30]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

# llm chain = off-the-shelf chain -> meaning general purpose chain -
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",  # make space for memory
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),# 시스템 / 사람/ AI 누가 어떤 양의 메세지를 보낼지 모르지만 MessagesPlaceholder 덕분에 Memory class로 대체 
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Mingi")

AttributeError: 'list' object has no attribute '_merge_partial_and_user_variables'

In [ ]:
chain.predict(question="I'm living in Singapore")

In [ ]:
chain.predict(question="What is my name?")